<a href="https://colab.research.google.com/github/MinorMole/RcloneLab/blob/master/RcloneLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img height="44px" src="https://rclone.org/img/rclone-120x120.png" align="left" hspace="20px"><h1>Welcome to RcloneLab!</h1><br>

- RcloneLab is a rclone GUI for Google Colaboratory. For more information please visit our [GitHub](https://github.com/MinorMole/RcloneLab/).



# Usage

In [0]:
# ============================= FORM ============================= #
#@title <center><h2>RcloneLab</h2></center> { display-mode: "form" }
Mode = 'Copy' #@param ["Config", "Copy", "Move", "Checker", "Deduplicate"]
Compare = 'Size & Mod-Time' #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = '' #@param {type:"string"}
Destination = '' #@param {type:"string"}
Transfers = 4 #@param {type:"slider", min:1, max:10, step:1}
Checkers = 8 #@param {type:"slider", min:1, max:20, step:1}
#@markdown <center><h2>Configuration</h2><br></center>
#@markdown <center><font color="red">* Required rclone.conf path. e.g. My Drive/rclone.conf</font></center>
Config = 'My Drive/rclone.conf' #@param {type:"string"}
#@markdown <center><h3><font color="#3399ff"><b>Global Configuration</b></font></h3></center>
Dry_Run = False #@param {type:"boolean"}
Log = False #@param {type:"boolean"}
Extra_Arguments = '' #@param {type:"string"}
#@markdown - [Read more...](https://rclone.org/commands/)
#@markdown <center><h3><font color="#3399ff"><b>Deduplicate Configuration</b></font></h3></center>
Deduplicate_Use_Trash = True #@param {type:"boolean"}
Deduplicate_Mode = 'Largest' #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
#@markdown - [Read more...](https://rclone.org/commands/rclone_dedupe/)
# ================================================================ #

# Variable Declaration

import os
from datetime import datetime,timedelta
os.environ['sourceC'] = Source
os.environ['destinationC'] = Destination
os.environ['transfersC'] = str(Transfers)
os.environ['checkersC'] = str(Checkers)
os.environ['configC'] = Config
os.environ['extraC'] = Extra_Arguments

if Compare == 'Size & Checksum':
  os.environ['compareC'] = "-c"
elif Compare == 'Only Mod-Time':
  os.environ['compareC'] = "--ignore-size"
elif Compare == 'Only Size':
  os.environ['compareC'] = "--size-only"
elif Compare == 'Only Checksum':
  os.environ['compareC'] = "-c --ignore-size"
else:
  os.environ['compareC'] = ""

if Dry_Run == True:
  os.environ['dryrunC'] = "-n"
else:
  os.environ['dryrunC'] = ""
  
if Deduplicate_Mode == 'Interactive':
  os.environ['deduplicateC'] = "interactive"
elif Deduplicate_Mode == 'Skip':
  os.environ['deduplicateC'] = "skip"
elif Deduplicate_Mode == 'First':
  os.environ['deduplicateC'] = "first"
elif Deduplicate_Mode == 'Newest':
  os.environ['deduplicateC'] = "newest"
elif Deduplicate_Mode == 'Oldest':
  os.environ['deduplicateC'] = "oldest"
elif Deduplicate_Mode == 'Largest':
  os.environ['deduplicateC'] = "largest"
elif Deduplicate_Mode == 'Rename':
  os.environ['deduplicateC'] = "rename"
  
if Deduplicate_Use_Trash == True:
  os.environ['deduplicatetrashC'] = ""
else:
  os.environ['deduplicatetrashC'] = "--drive-use-trash=false"

# Code Execution

if Config == '':
  print("Please config the rclone.conf in Setup section first.")
elif Mode == 'Config':
  !rclone --config=/GD/"$configC" config
elif Log == True:
  if Mode == 'Copy':
    !rclone --config=/GD/"$configC" copy "$sourceC" "$destinationC" --transfers $transfersC --checkers $checkersC -v --stats=5s $compareC $dryrunC $extraC --log-file=rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
  elif Mode == 'Move':
    !rclone --config=/GD/"$configC" move "$sourceC" "$destinationC" --transfers $transfersC --checkers $checkersC -v --stats=5s --delete-empty-src-dirs --drive-keep-revision-forever $compareC $dryrunC $extraC --log-file=rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
  elif Mode == 'Checker':
    !rclone --config=/GD/"$configC" check "$sourceC" "$destinationC" --checkers $checkersC -v --stats=5s $compareC $dryrunC $extraC --log-file=rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
  elif Mode == 'Deduplicate':
    !rclone --config=/GD/"$configC" dedupe "$sourceC" --checkers $checkersC -v --stats=5s --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $dryrunC $extraC --log-file=rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
else:
  if Mode == 'Copy':
    !rclone --config=/GD/"$configC" copy "$sourceC" "$destinationC" --transfers $transfersC --checkers $checkersC -v --stats=5s $compareC $dryrunC $extraC
  elif Mode == 'Move':
    !rclone --config=/GD/"$configC" move "$sourceC" "$destinationC" --transfers $transfersC --checkers $checkersC -v --stats=5s --delete-empty-src-dirs --drive-keep-revision-forever $compareC $dryrunC $extraC
  elif Mode == 'Checker':
    !rclone --config=/GD/"$configC" check "$sourceC" "$destinationC" --checkers $checkersC -v --stats=5s $compareC $dryrunC $extraC
  elif Mode == 'Deduplicate':
    !rclone --config=/GD/"$configC" dedupe "$sourceC" --checkers $checkersC -v --stats=5s --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $dryrunC $extraC
    
if Log == True and Mode != "Config":
  os.environ['logPathC'] = "/".join(Config.split("/")[:-1])
  !rclone --config=/GD/"$configC" move *.txt "/GD/""$logPathC""/" --transfers 1 --checkers 1 --max-depth 1 -c -q
  !echo Your log will be ready at \"$logPathC\"

# Installation

In [0]:
#@markdown <h3>Note: Run this before use.</h3> { display-mode: "form" }
Setup_Time_Zone = True #@param {type:"boolean"}
rclone_Beta = False #@param {type:"boolean"}

if rclone_Beta == True:
  !curl https://rclone.org/install.sh | sudo bash -s beta
else:
  !curl https://rclone.org/install.sh | sudo bash
if Setup_Time_Zone == True:
  !sudo dpkg-reconfigure tzdata
from google.colab import drive
drive.mount('/GD')